## This Nb does some cleansing and transformations over the silver table as a pre-step prior to creating the gold table
### It is an interesting exercise the learn and practice pyspark operations

In [0]:
import pandas as pd

# Data cleaning
df = spark.sql("select * from job_catalog.default.silver_customers_sales")

# Strip whitespaces from columns names
df = df.toDF(*[c.strip() for c in df.columns])

# Convert data type from string to date
df = df.withColumn("order_date", df.order_date.cast("date"))

# Drop rows with missing values
df = df.dropna()

from pyspark.sql.functions import col, year, month, when

# Extract year and month from order_date
df = df.withColumn("order_year", year(col("order_date"))).withColumn("order_month", month(col("order_date")))

# Map month numbers to month names using when/otherwise
df = df.withColumn(
    "order_month",
    when(col("order_month") == 1, "January")
    .when(col("order_month") == 2, "February")
    .when(col("order_month") == 3, "March")
    .when(col("order_month") == 4, "April")
    .when(col("order_month") == 5, "May")
    .when(col("order_month") == 6, "June")
    .when(col("order_month") == 7, "July")
    .when(col("order_month") == 8, "August")
    .when(col("order_month") == 9, "September")
    .when(col("order_month") == 10, "October")
    .when(col("order_month") == 11, "November")
    .when(col("order_month") == 12, "December")
    .otherwise(None)
)

display(df)

In [0]:
df.write.mode("overwrite").saveAsTable("job_catalog.default.gold_customers_sales")